In [ ]:
import numpy as np
import cv2
import time
import imutils


def draw_flow(img, flow, step=16):

    h, w = img.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)
    fx, fy = flow[y,x].T

    lines = np.vstack([x, y, x-fx, y-fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)

    img_bgr = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    cv2.polylines(img_bgr, lines, 0, (0, 255, 0))

    for (x1, y1), (_x2, _y2) in lines:
        cv2.circle(img_bgr, (x1, y1), 1, (0, 255, 0), -1)

    return img_bgr


def draw_hsv(flow):

    h, w = flow.shape[:2]
    fx, fy = flow[:,:,0], flow[:,:,1]

    ang = np.arctan2(fy, fx) + np.pi
    v = np.sqrt(fx*fx+fy*fy)

    hsv = np.zeros((h, w, 3), np.uint8)
    hsv[...,0] = ang*(180/np.pi/2)
    hsv[...,1] = 255
    hsv[...,2] = np.minimum(v*4, 255)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    return bgr

video_path = '.\\videos\\video2.mov'
cap = cv2.VideoCapture(video_path)

suc, prev = cap.read()
prevgray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
valor = 600
prevgray = imutils.resize(prevgray, width=min(valor, prevgray.shape[1]))

while True:
    suc, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = imutils.resize(gray, width=min(valor, gray.shape[1]))
    # start time to calculat FPS
    start = time.time()
    flow = cv2.calcOpticalFlowFarneback(prevgray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    prevgray = gray
    # End time
    end = time.time()
    # calculate the FPS for current frame detection
    fps = 1 / (end-start)
    print(f"{fps:.2f} FPS")
    cv2.imshow('flow', draw_flow(gray, flow))
    cv2.imshow('flow HSV', draw_hsv(flow))
    key = cv2.waitKey(5)
    if key == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [1]:
import cv2
import numpy as np
import time
import imutils

def draw_flow(img, p0, p1, mask=None, color=(0, 255, 0), line_thickness=2):
    for i, (new, old) in enumerate(zip(p1, p0)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), color, line_thickness)
        img = cv2.circle(img, (int(a), int(b)), 5, color, -1)
    return cv2.add(img, mask)

video_path = '.\\videos\\video2.mp4'
cap = cv2.VideoCapture(video_path)

suc, prev = cap.read()
prevgray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
valor = 600
prevgray = imutils.resize(prevgray, width=min(valor, prevgray.shape[1]))


feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
p0 = cv2.goodFeaturesToTrack(prevgray, mask=None, **feature_params)

mask = np.zeros_like(prev)

while True:
    suc, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = imutils.resize(gray, width=min(valor, gray.shape[1]))

    start = time.time()
    p1, st, err = cv2.calcOpticalFlowPyrLK(prevgray, gray, p0, None, winSize=(15, 15), maxLevel=2, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
    end = time.time()

    if end != start:
        fps = 1 / (end - start)
        print(f"{fps:.2f} FPS")
    else:
        print("Unable to calculate FPS (division by zero)")

    img = draw_flow(img, p0, p1, mask=mask)

    cv2.imshow('flow', img)
    p0 = p1.reshape(-1, 1, 2)

    key = cv2.waitKey(5)
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


663.97 FPS
990.86 FPS
997.93 FPS
993.68 FPS
997.22 FPS
998.41 FPS
504.18 FPS
249.91 FPS
496.31 FPS
1009.46 FPS
499.44 FPS
999.36 FPS
1039.48 FPS
497.37 FPS
1036.91 FPS
488.96 FPS
997.46 FPS
498.91 FPS
1000.31 FPS
1001.51 FPS
998.88 FPS
992.50 FPS
992.97 FPS
661.25 FPS
496.90 FPS
993.44 FPS
1036.40 FPS
998.64 FPS
1002.70 FPS
993.44 FPS
978.83 FPS
1000.31 FPS
1034.35 FPS
1014.34 FPS
994.85 FPS
693.73 FPS
997.46 FPS
994.62 FPS
997.22 FPS
1002.94 FPS
997.46 FPS
1040.00 FPS
832.70 FPS
1003.66 FPS
998.17 FPS
990.16 FPS
982.27 FPS
993.20 FPS
998.64 FPS
663.34 FPS
217.57 FPS
998.88 FPS
199.59 FPS
378.04 FPS
997.93 FPS
179.80 FPS
998.64 FPS
500.99 FPS
995.80 FPS
382.10 FPS
992.73 FPS
996.75 FPS
965.76 FPS
503.46 FPS
248.45 FPS
1003.18 FPS
509.08 FPS
493.39 FPS
492.00 FPS
247.29 FPS
992.97 FPS
502.19 FPS
250.92 FPS
331.85 FPS
520.06 FPS
502.61 FPS
476.73 FPS
1016.31 FPS
248.42 FPS
504.79 FPS
1017.79 FPS
250.27 FPS
498.97 FPS
1001.98 FPS
1003.90 FPS
249.53 FPS
1009.46 FPS
388.65 FPS
329.77 FPS
10